In [1]:
from datasets import load_dataset

SPLIT = "test" # ["train", "val", "test"]
FINAL_OUPUT = f"final_data_{SPLIT}.txt"


ds = load_dataset("OpenSound/CapTTS-SFT", split=SPLIT)
df = ds.to_pandas()
n_origin = len(df)
df.head()

/home/nhandt23/anaconda3/envs/speechcraft/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,source,audio_path,caption,text,speech_duration,accent,emotion,speaking_rate,intrinsic_tags,emotion_text_prompt,all_tags,age,pitch,situational_tags,gender,basic_tags,noise,speech_monotony
0,vctk,p326_358.wav,"A mature, Australian male voice, marked by a d...",Labour is providing none of these.,3.628542,australian,None,None,None,None,None,None,None,None,None,None,None,None
1,vctk,p264_235.wav,"A weak, teenage girl voice with a Scottish acc...","After the match, do you?",3.671000,scottish,None,None,None,None,None,None,None,None,None,None,None,None
2,vctk,p376_055.wav,The voice of an adult Indian man.,I'm looking forward to the next two weeks.,3.114208,indian,None,None,None,None,None,None,None,None,None,None,None,None
3,vctk,s5_398.wav,"A mature British female voice, bright and smoo...",That will be a decision for BBC Scotland.,3.372333,british,None,None,None,None,None,None,None,None,None,None,None,None
4,vctk,p251_029.wav,An elderly Indian man with a nasal and weak vo...,"However, the following year the cancer returned.",3.267292,indian,None,None,None,None,None,None,None,None,None,None,None,None


In [2]:
df["source"].unique()

array(['vctk', 'emovoice-db', 'voxceleb', 'ears', 'expresso',
       'libritts-r'], dtype=object)

## Expresso

In [3]:
import pandas as pd
import os

base_dir_read = '/data1/speech/nhandt23/06_binh/dataset/expresso/audio_48khz/read/'

is_conversational = (df['source'] == 'expresso') & \
                    (df['audio_path'].str.split('_').str[0].str.contains('-'))

df_clean = df[~is_conversational].copy()

def get_read_path(filename):
    try:
        parts = filename.split('_')
        speaker = parts[0] # ex01
        emotion = parts[1] # happy
        # Ghép: base_dir / speaker / emotion / base / filename
        return os.path.join(base_dir_read, speaker, emotion, 'base', filename)
    except:
        return filename

df_clean['audio_path'] = df_clean.apply(
    lambda row: get_read_path(row['audio_path']) if row['source'] == 'expresso' else row['audio_path'], 
    axis=1
)
print(f"Số lượng dòng ban đầu: {len(df)}")
print(f"Số lượng dòng sau khi lọc: {len(df_clean)}")
""

Số lượng dòng ban đầu: 20756
Số lượng dòng sau khi lọc: 19886


''

## LibriTTS-R

In [4]:
libri_dir = "/data1/speech/cuongph/01_database/librittsr/LibriTTS_R"
df_clean["audio_path"] = df_clean.apply(
    lambda row: os.path.join(libri_dir, row["audio_path"]) if row["source"] == "libritts-r" else row["audio_path"],
    axis=1
)

## VoxCeleb

In [5]:
import os
from tqdm import tqdm

voxceleb_dir2 = "/data1/speech/kienpt/data/speaker_verify/train_VoxCeleb2/wavs"
voxceleb_dir1 = "/data1/speech/speechData/rawData/Public_Data/VOXCELEB1/wav"
rm_idx = []

target_df = df_clean[df_clean["source"] == "voxceleb"]

for idx, row in tqdm(target_df.iterrows(), total=target_df.shape[0]):
    audio_path = row["audio_path"] # Ví dụ: id10230-WxmrMgdkqOw-00004.wav
    parts = audio_path.split("-")
    
    # parts[0] = id10230
    # parts[1] = WxmrMgdkqOw
    # parts[2] = 00004.wav (Đã có sẵn .wav nếu tên file gốc có)
    
    filename = parts[2]
    if not filename.endswith(".wav"):
        filename += ".wav"
        
    wavs_path2 = os.path.join(voxceleb_dir2, parts[0], parts[1], filename)
    wavs_path1 = os.path.join(voxceleb_dir1, parts[0], parts[1], filename)
    
    if not os.path.exists(wavs_path2) and not os.path.exists(wavs_path1):
        if len(rm_idx) < 3:
            print(f"Missing: {audio_path}")
        rm_idx.append(idx)

print(f"Tổng số file thiếu: {len(rm_idx)}")
df_clean = df_clean.drop(rm_idx)
df_clean = df_clean.reset_index(drop=True)

  2%|▏         | 210/12551 [00:00<00:11, 1058.32it/s]

Missing: id06692-GDszoDRymKo-00239.wav
Missing: id00255-ydz-WMJFtBc-00500.wav
Missing: id04964-Cv2V-OqS0gU-00024.wav


100%|██████████| 12551/12551 [00:07<00:00, 1781.97it/s]

Tổng số file thiếu: 1910


In [6]:
import os

voxceleb_dir = "/data1/speech/kienpt/data/speaker_verify/train_VoxCeleb2/wavs"

def make_absolute_path(row):
    if row["source"] == "voxceleb":
        parts = row["audio_path"].split("-")
        if len(parts) >= 3:
            filename = parts[2]
            if not filename.endswith(".wav"):
                filename += ".wav"
            return os.path.join(voxceleb_dir, parts[0], parts[1], filename)
    
    return row["audio_path"]

df_clean["audio_path"] = df_clean.apply(make_absolute_path, axis=1)

print("Sample path sau khi update:")
print(df_clean[df_clean["source"] == "voxceleb"]["audio_path"].iloc[0])

Sample path sau khi update:
/data1/speech/kienpt/data/speaker_verify/train_VoxCeleb2/wavs/id03460/RFLQrhgyU2Q/00171.wav


## ears

In [7]:
import os

ears_path = "/data1/speech/nhandt23/06_thang/ears_dataset"

def make_absolute_path(row):
    if row["source"] == "ears":
        parts = row["audio_path"].split("-")
        return os.path.join(ears_path, parts[0], parts[1])
    
    return row["audio_path"]

df_clean["audio_path"] = df_clean.apply(make_absolute_path, axis=1)

print("Sample path sau khi update:")
print(df_clean[df_clean["source"] == "ears"]["audio_path"].iloc[0])

Sample path sau khi update:
/data1/speech/nhandt23/06_thang/ears_dataset/p092/emo_guilt_freeform.wav


## Emovoice-db

In [8]:
df_clean[df_clean["source"] == "emovoice-db"].iloc[0]["audio_path"]

'gpt4o_19981_surprised_verse.wav'

In [9]:
emodb_path = "/data1/speech/speechData/rawData/Public_Data/EmoVoice-DB/audio"

df_clean["audio_path"] = df_clean.apply(
    lambda row: os.path.join(emodb_path, row["audio_path"]) if row["source"] == "emovoice-db" else row["audio_path"],
    axis=1
)

## vctk

In [10]:
len(df_clean[df_clean["source"] == "vctk"])

500

In [11]:
import os
vctk_path = "/data1/speech/speechData/rawData/Public_Data/VCTK/VCTK-Corpus/wav48"

def make_absolute_path(row):
    if row["source"] == "vctk":
        parts = row["audio_path"].split("_")
        return os.path.join(vctk_path, parts[0], row["audio_path"])
    
    return row["audio_path"]

df_clean["audio_path"] = df_clean.apply(make_absolute_path, axis=1)

print("Sample path sau khi update:")
print(df_clean[df_clean["source"] == "vctk"]["audio_path"].iloc[0])

Sample path sau khi update:
/data1/speech/speechData/rawData/Public_Data/VCTK/VCTK-Corpus/wav48/p326/p326_358.wav


In [12]:
# check

def check_exist(path):
    return os.path.exists(path)

df_clean["exists"] = df_clean["audio_path"].apply(check_exist)
print(df_clean[df_clean["exists"] == 0].groupby("source").size())
df_clean = df_clean[df_clean["exists"] != 0].reset_index(drop=True)

source
vctk          45
voxceleb    2199
dtype: int64


In [13]:
len(df_clean)

15732

## Export

In [14]:
df_clean.head()

,source,audio_path,caption,text,speech_duration,accent,emotion,speaking_rate,intrinsic_tags,emotion_text_prompt,all_tags,age,pitch,situational_tags,gender,basic_tags,noise,speech_monotony,exists
0,vctk,/data1/speech/speechData/rawData/Public_Data/V...,"A mature, Australian male voice, marked by a d...",Labour is providing none of these.,3.628542,australian,None,None,None,None,None,None,None,None,None,None,None,None,True
1,vctk,/data1/speech/speechData/rawData/Public_Data/V...,"A weak, teenage girl voice with a Scottish acc...","After the match, do you?",3.671000,scottish,None,None,None,None,None,None,None,None,None,None,None,None,True
2,vctk,/data1/speech/speechData/rawData/Public_Data/V...,The voice of an adult Indian man.,I'm looking forward to the next two weeks.,3.114208,indian,None,None,None,None,None,None,None,None,None,None,None,None,True
3,vctk,/data1/speech/speechData/rawData/Public_Data/V...,An elderly Indian man with a nasal and weak vo...,"However, the following year the cancer returned.",3.267292,indian,None,None,None,None,None,None,None,None,None,None,None,None,True
4,vctk,/data1/speech/speechData/rawData/Public_Data/V...,"A mature male voice, characterized by a deep a...","I saw the story, but that amount wouldn't even...",4.797333,english,None,None,None,None,None,None,None,None,None,None,None,None,True


In [15]:
print("Số hàng ban đầu:", n_origin)
print("Số hàng sau khi filter:", len(df_clean))

df_clean[["audio_path","text" ,"caption"]].to_csv(FINAL_OUPUT, sep="|", header=None, index=False)

Số hàng ban đầu: 20756
Số hàng sau khi filter: 15732
